In [ ]:
!pip install pydotplus
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 57.0 MB/s eta 0:00:00


In [ ]:
# install the dependencies
# Python ≥3.5 is required

import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
#assert tf.__version__ >= "2.0"
from tensorflow import keras

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)

import time

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from IPython.display import clear_output

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)





Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# load the dataset
dataset_url = "https://raw.githubusercontent.com/lee-anh/FlightPredictions/main/flightsResult1.csv"
dataset_url1 =  "https://raw.githubusercontent.com/lee-anh/FlightPredictions/main/flightsResult2.csv"
df = pd.read_csv(dataset_url)
df = df.append(pd.read_csv(dataset_url1))

# look at the headers
data_top = df.head() 
data_top

<ipython-input-27-84f94f07ecbf>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(dataset_url1))


,Unnamed: 0,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,YEAR,MONTH,...,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL
0,0,14.54,0.0,0.0,0.0,20,32,14,2015,1,...,US,602,N197UW,ORD,PHX,500,-1.0,228.0,1440,748
1,1,14.54,0.0,0.0,0.0,20,32,14,2015,1,...,UA,1500,N30401,ORD,IAH,510,4.0,175.0,925,805
2,2,14.54,0.0,0.0,0.0,20,32,14,2015,1,...,NK,409,N509NK,ORD,FLL,530,-4.0,188.0,1182,938
3,3,14.54,0.0,0.0,0.0,20,32,14,2015,1,...,UA,1167,N66837,ORD,DEN,533,7.0,167.0,888,720
4,4,14.54,0.0,0.0,0.0,20,32,14,2015,1,...,EV,5498,N132EV,ORD,DTW,535,15.0,85.0,235,800


In [ ]:

"""
# data preprocessing

# ONE HOT ENCODING TUTORIAL HERE https://www.geeksforgeeks.org/ml-one-hot-encoding-of-datasets-in-python/
# ADDS EXTRA COLUMN FOR NUMBER CORRESPONDING TO ONE HOT ENCODING WILL DROP THIS
# these two make it like 83 
df['Condition'] = df['Condition'].astype('category')
df['DEST'] = df['DEST'].astype('category')

# these two one hots make it 2k 
df['OP_UNIQUE_CARRIER'] = df['OP_UNIQUE_CARRIER'].astype('category')
# df['TAIL_NUM'] = df['TAIL_NUM'].astype('category')
  
  
# Assigning numerical values and storing it in another columns
df['Gen_new'] = df['Condition'].cat.codes
df['Rem_new'] = df['DEST'].cat.codes

df['Car_new'] = df['OP_UNIQUE_CARRIER'].cat.codes
  
  
# Create an instance of One-hot-encoder
enc = OneHotEncoder()
  
# Passing encoded columns
enc_data = pd.DataFrame(enc.fit_transform(
    df[['Gen_new', 'Rem_new','Car_new']]).toarray())
  
# Merge with main
New_df = df.join(enc_data)

# dropping extra columns
New_df = New_df.drop("Gen_new", axis='columns')
New_df = New_df.drop("Rem_new", axis='columns')
New_df = New_df.drop("Car_new", axis='columns')
New_df = New_df.drop("Condition", axis='columns')
New_df = New_df.drop("DEST", axis='columns')
New_df = New_df.drop("OP_UNIQUE_CARRIER", axis='columns')
New_df = New_df.drop("Wind", axis='columns')

# TODO: group planes by number of delays
New_df = New_df.drop("TAIL_NUM", axis='columns')



# scaling pressure by 100 so that the floats don't get truncated when we convert to ints

# I think we need to convert all non integer types into integers
# meow meow meow 
carrier_cats = sorted(df["OP_UNIQUE_CARRIER"].unique())
dest_cats = sorted(df["DEST"].unique())
wind_cats = sorted(df["Wind"].astype(str).unique()) # there is a nan value, we could remove that data point
condition_cats = sorted(df["Condition"].unique())

# convert to dictionaries ... we might not need this anymore 
carrier_dict = dict(enumerate(carrier_cats)) 
carrier_dict = {y: x for x, y in carrier_dict.items()} # flip the keys and values 
dest_dict = dict(enumerate(dest_cats)) 
dest_dict = {y: x for x, y in dest_dict.items()}
wind_dict = dict(enumerate(wind_cats)) 
wind_dict = {y: x for x, y in wind_dict.items()}
condition_dict = dict(enumerate(condition_cats)) 
condition_dict = {y: x for x, y in condition_dict.items()}


# apply the maps to convert the data
vocab = carrier_cats + dest_cats + wind_cats + condition_cats
assert len(vocab) == len(set(vocab)) # cool we have a unqiue string vocab  

"""

'\n# I think we need to convert all non integer types into integers\n# meow meow meow \ncarrier_cats = sorted(df["OP_UNIQUE_CARRIER"].unique())\ndest_cats = sorted(df["DEST"].unique())\nwind_cats = sorted(df["Wind"].astype(str).unique()) # there is a nan value, we could remove that data point\ncondition_cats = sorted(df["Condition"].unique())\n\n# convert to dictionaries ... we might not need this anymore \ncarrier_dict = dict(enumerate(carrier_cats)) \ncarrier_dict = {y: x for x, y in carrier_dict.items()} # flip the keys and values \ndest_dict = dict(enumerate(dest_cats)) \ndest_dict = {y: x for x, y in dest_dict.items()}\nwind_dict = dict(enumerate(wind_cats)) \nwind_dict = {y: x for x, y in wind_dict.items()}\ncondition_dict = dict(enumerate(condition_cats)) \ncondition_dict = {y: x for x, y in condition_dict.items()}\n\n\n# apply the maps to convert the data\nvocab = carrier_cats + dest_cats + wind_cats + condition_cats\nassert len(vocab) == len(set(vocab)) # cool we have a unqiue

In [ ]:
# y 
import tensorflow_decision_forests as tfdf
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.tree import export_graphviz
from sklearn import tree
from six import StringIO
import pydotplus
from IPython.display import Image 
dot_data = StringIO()
from sklearn import tree
import xgboost as xgb


#importing the data



df.columns = df.columns.astype(str)

delay_times = df.loc[:, "DEPARTURE_DELAY"]
df = df.drop(["DEPARTURE_DELAY"], axis="columns")
number_of_classes = 1 
y = delay_times.map(lambda a : 1 if a > 0 else 0 ) # using a binary classifer right now, but will change 

print(y)


# drop features that are unimportant / would potentially confound the data


# X
X = df
number_of_features = df.shape[1]


# split the dataset to train, validation, and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 1)

# simple binary classifer

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# Changing data to int

#X_train=np.asarray(X_train).astype(int)
#y_train=np.asarray(y_train).astype(int)

#X_val=np.asarray(X_val).astype(int)
#y_val=np.asarray(y_val).astype(int)

"""
#DECISION TREE MODEL
#Decision tree model, does the best out of everything, but can be made better
clf = XGBClassifier(random_state = 0)
clf.fit(X_train, y_train) 

tree.export_graphviz(clf, max_depth = 3, feature_names=X_test.columns, out_file= dot_data)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 
graph.write_png('tree.png')
# image stored in files
Image(graph.create_png())



print(X_test)
y_predict = clf.predict(X_test)
cmatrix = confusion_matrix(y_test, y_predict) 
accuracy = accuracy_score(y_test, y_predict) 
classification_report = classification_report(y_test, y_predict) 

print(cmatrix)
print(classification_report)


"""

#tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(df, label="DEPARTURE_DELAY")

tuner = tfdf.tuner.RandomSearch(num_trials=20)

# Hyper-parameters to optimize.
#tuner.discret("max_depth", [4, 5, 6, 7])

model = tfdf.keras.GradientBoostedTreesModel() #tuner=tuner
model.fit(X_train, y_train)

print(model.summary())



# we could try different things like logisitic, then softmax, then cnn and then with cnn we can do hyper parameter tuning 

0         0
1         1
2         0
3         1
4         1
         ..
145879    0
145880    1
145881    0
145882    0
145883    1
Name: DEPARTURE_DELAY, Length: 285884, dtype: int64
Use /tmp/tmp85b95oof as temporary training directory


ValueError: ignored

The Decision Tree has excellent performance. However, Decision Trees are unstable and for this model to be generalizable to other airports we will need to use a more stable model, like a Neural Network. 

In [ ]:
# DNN MODEL as a binary classifier 

simple_model = keras.models.Sequential()

simple_model.add(keras.layers.Dense(100, activation = 'tanh'))
simple_model.add(keras.layers.Dense(50, activation = 'tanh'))
simple_model.add(keras.layers.Dense(15, activation = 'tanh'))
simple_model.add(keras.layers.Dense(5, activation = 'relu'))
simple_model.add(keras.layers.Dense(1, activation = 'sigmoid'))

simple_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])


simple_model.summary
simple_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))




Our DNN has low variance but high bias which means that the model is underfitting. We will try to use a more complex model. 

In [ ]:
# model 2 DNN


model_2 = keras.models.Sequential()
for x in range(500): 
    model_2.add(keras.layers.Dense(256, activation = 'relu'))


model_2.add(keras.layers.Dense(5, activation = "relu"))
model_2.add(keras.layers.Dense(1, activation = 'sigmoid'))
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])


model_2.summary

num_epochs = 10

tic = time.process_time()
model_2_history = model_2.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val))
toc = time.process_time()


#Plot performnace
pd.DataFrame(model_2_history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

time_nn_f=toc-tic
print("Time to train the model: {0:.4f} secs ".format((time_nn_f)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(model_2_history.history['accuracy'][num_epochs-1],model_2_history.history['loss'][num_epochs-1]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(model_2_history.history['val_accuracy'][num_epochs-1],model_2_history.history['val_loss'][num_epochs-1]))


# even when we add a bunchhhhh of layers we are underfitting... time to look for better features 


We might need to improve our data to have more features

In [ ]:
# now trying with xgboost 